# Analyzing borrowers’ risk of defaulting

Your project is to prepare a report for a bank’s loan division. You’ll need to find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness.

Your report will be considered when building the **credit score** of a potential customer. The **credit score** is used to evaluate the ability of a potential borrower to repay their loan.

---
The purpose of this project is to determine whether a customer's marital status and the number of children in their family will affect a customer's risk of defaulting on a loan. The initial hypotheses tested through this analysis are the following:

1) Married customers will have a lower risk of defaulting on a loan than unmarried customers

2) Customers with no children will have a lower risk of defaulting on a loan than customers with one or more children

## Open the data file and have a look at the general information. 

In [1]:
# Load the libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")


In [2]:
# Load nltk library to utilize lemmatizer
import nltk
from nltk.stem import WordNetLemmatizer
wordnet_lemma = WordNetLemmatizer()


In [3]:
# Load the data as 'df'
df = pd.read_csv('/datasets/credit_scoring_eng.csv')


To begin, the pandas, numpy, warnings, nltk, and WordNetLemmatizer libraries will be imported. The dataset will be loaded as 'df'.

## Task 1. Data exploration

**Description of the data**
- `children` - the number of children in the family
- `days_employed` - work experience in days
- `dob_years` - client's age in years
- `education` - client's education
- `education_id` - education identifier
- `family_status` - marital status
- `family_status_id` - marital status identifier
- `gender` - gender of the client
- `income_type` - type of employment
- `debt` - was there any debt on loan repayment
- `total_income` - monthly income
- `purpose` - the purpose of obtaining a loan

In [4]:
# Check the number of rows and columns in the dataset 'df'
original_shape = df.shape
print(original_shape)


(21525, 12)


Looking at the info printed above in the tuple, there are 21,525 rows and 12 columns.

In [5]:
# Print the first 20 rows of 'df'
print(df.head(20))


    children  days_employed  dob_years            education  education_id  \
0          1   -8437.673028         42    bachelor's degree             0   
1          1   -4024.803754         36  secondary education             1   
2          0   -5623.422610         33  Secondary Education             1   
3          3   -4124.747207         32  secondary education             1   
4          0  340266.072047         53  secondary education             1   
5          0    -926.185831         27    bachelor's degree             0   
6          0   -2879.202052         43    bachelor's degree             0   
7          0    -152.779569         50  SECONDARY EDUCATION             1   
8          2   -6929.865299         35    BACHELOR'S DEGREE             0   
9          0   -2188.756445         41  secondary education             1   
10         2   -4171.483647         36    bachelor's degree             0   
11         0    -792.701887         40  secondary education             1   

At first glance based on the printed data sample above, the following issues should be noted and further investigated:

1) Most of the values in the 'days_employed' column are negative float values while the positive values are inplausible

2) The values in the 'education" column are not standardized; there are mainly inconsistencies in the capitalization of the string values

3) There are missing values in the 'days_employed' and 'total_income' columns

In [6]:
# Get info on data and find the total number of missing values in the dataset
df.info()
df.isna().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Based on the information above, the missing values that are present in the dataset are only present in the 'days_employed' and 'total_income' columns; 2,174 missing values in each to be exact.

In [7]:
# Look at a filtered table with missing values in the 'days_employed' column
filtered_df = df[df['days_employed'].isna()]
display(filtered_df)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


In [8]:
# Apply multiple conditions for filtering the data (missing values in the 'days_employed' and 'total_income' columns) and look at the number of rows in the filtered table.
multiple_filtered_df = df[(df['days_employed'].isna()) & (df['total_income'].isna())]
display(multiple_filtered_df)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


Print and examine filtered tables that show all the rows with missing data first in just the 'days_employed' column and then in the 'days_employed' and 'total_income" column to check if they occur concurrently.

**Intermediate conclusion**

Based on the filtered tables shown above, it appears that the missing values always appear together in the same rows and are thus symmetric. We can confirm this as the number of rows in the filtered table matches the total number of missing values.

A potential reason for this is because the 'days_employed' and 'total_income" columns are both related to a customer's occupation. If the individual is not currently employed at the time of application, these two values may not have been submitted since they are not applicable to the customer at the time. As such, I would hypothesize that the 'income_type' for the customers with missing data would largely fall under the 'retiree' category.

In [9]:
# Calculate the percentage of missing values compared to the whole dataset
percent_missing = multiple_filtered_df.shape[0] / df.shape[0] * 100
display(percent_missing)


10.099883855981417

Since the percentage of rows with missing values compared to the whole dataset is approximately 10%, I would conclude that as this is a rather large piece of missing data, it will be important to further investigate whether it is affected by other specific client characteristics and check whether there is any dependence that the missing values have on them. Below, I will specifically investigate my aforementioned hypothesis that the 'income_type' client characteristic for the customers with missing data will largely fall under the 'retiree' category.

In [10]:
# Investigate clients with missing data in 'days_employed' and 'total_income' by checking the counts in each income type
multiple_filtered_df['income_type'].value_counts()


employee         1105
business          508
retiree           413
civil servant     147
entrepreneur        1
Name: income_type, dtype: int64

In [11]:
# Checking the distribution of income types for the rows with missing values
multiple_filtered_df['income_type'].value_counts()/ multiple_filtered_df.shape[0] * 100


employee         50.827967
business         23.367065
retiree          18.997240
civil servant     6.761730
entrepreneur      0.045998
Name: income_type, dtype: float64

When I examined the percentage of each potential categorical value in the 'income_type' column for customers that had missing values, I discovered that the percentage of retirees (19%) is actually lower than that of employees (51%) and business owners (23%). Therefore, I can conclude that my hypothesis that that the 'income_type' client characteristic for customers with missing data does not largely fall under the 'retiree' category.

**Possible reasons for missing values in data**

Given the results of the above analysis, it appears that there may be another reason for why there is missing data for some customers. I would now hypothesize that these missing data points are actually random. It is likely that some customers randomly chose not to provide the number of days that they were employed and their current income. It is also very possible that the questions associated with these two columns were somehow linked (i.e. if a customer did not fill out the number of days that they were employed, it automatically caused the total income category to be skipped as well). To confirm this hypothesis, I will examine the distribution of income types across the entire dataset to compare with the distribution found only among those with missing data.

In [12]:
# Check the distribution of income types in the whole dataset
df['income_type'].value_counts() / df.shape[0] * 100


employee                       51.656214
business                       23.623693
retiree                        17.914053
civil servant                   6.778165
entrepreneur                    0.009292
unemployed                      0.009292
student                         0.004646
paternity / maternity leave     0.004646
Name: income_type, dtype: float64

**Intermediate conclusion**

When I examined the overall distribution of 'income_type' categories in the overall dataset, I found similar percentages for each category which indicates that the missing values are in fact not dependent on the 'income_type' after all. As such, it seems reasonable to conclude that the missing values have been randomly assigned.

**Conclusions**

Through this initial exploration of the data, it seems clear that there are missing values that comprise a significant portion of the data that should be addressed prior to analysis. These missing values appear to have been randomly assigned and occur concurrently in the 'days_employed' and 'total_income' column. They will be filled using information from the non-missing data in the columns as well as the other variables available in the dataset. Prior to filling the missing values, the data will be transformed to address duplicates, register inconsistencies, and fix any incorrect artifacts to ensure that the data is properly cleaned for formal analysis.

## Data transformation

In [13]:
# Examine unique values in the 'education' column to check if and what spellings and registers will need to be fixed
df['education'].unique()


array(["bachelor's degree", 'secondary education', 'Secondary Education',
       'SECONDARY EDUCATION', "BACHELOR'S DEGREE", 'some college',
       'primary education', "Bachelor's Degree", 'SOME COLLEGE',
       'Some College', 'PRIMARY EDUCATION', 'Primary Education',
       'Graduate Degree', 'GRADUATE DEGREE', 'graduate degree'],
      dtype=object)

The unique 'education" values indicate that there inconsistencies in the capitalization of the words which need to be fixed.

In [14]:
# Fix the inconsistent registers
df['education'] = df['education'].str.lower()


The string values in the 'education' column were made all lowercase to prevent duplicates due to capitalization issues.

In [15]:
# Check all the unique values in the 'education' column again to make sure they are fixed
df['education'].unique()


array(["bachelor's degree", 'secondary education', 'some college',
       'primary education', 'graduate degree'], dtype=object)

When the unique values in the 'education' column were checked again, all the duplicates were removed.

In [16]:
# Check the distribution of values in the 'children' column
df['children'].value_counts() / df.shape[0] * 100


 0     65.732869
 1     22.383275
 2      9.547038
 3      1.533101
 20     0.353078
-1      0.218351
 4      0.190476
 5      0.041812
Name: children, dtype: float64

Next, the distribution of values in the 'children' column was displayed which showed a few potential errors in the data. The main values that are of concern are -1, which is impossible since you cannot have a negative number of children, and 20, which is a number far too high to occur so frequently in one dataset. These problematic values may be a result of inputting errors or a glitch in the system used to record answers. But since they only comprise a minor percentage of the data (0.2% and 0.4% respectively), it seems appropriate to simply drop the values from the dataset so the erroneous values do not create issues in the remainder of the analysis.

In [17]:
# Drop the rows where '-1' or '20' were inputted as values in the 'children' column
df = df[(df['children'] >= 0) & (df['children'] <= 5)]


In [18]:
# Check the unique values in the 'children' column to make sure the drop was successful
print(sorted(df['children'].unique()))


[0, 1, 2, 3, 4, 5]


As shown above, the rows with values -1 or 20 in the 'children' column were dropped from the dataset. The unique values in the 'children' column were printed in order to ensure that the drop was successful. Since -1 or 20 do not appear in the list of unique values, we can infer that it was.

In [19]:
# Examine some rows in the 'days_employed' column to get a sense of what may be problematic
df['days_employed'].head(20)


0      -8437.673028
1      -4024.803754
2      -5623.422610
3      -4124.747207
4     340266.072047
5       -926.185831
6      -2879.202052
7       -152.779569
8      -6929.865299
9      -2188.756445
10     -4171.483647
11      -792.701887
12              NaN
13     -1846.641941
14     -1844.956182
15      -972.364419
16     -1719.934226
17     -2369.999720
18    400281.136913
19    -10038.818549
Name: days_employed, dtype: float64

In [20]:
# Find problematic data in the 'days_employed' and calculate the percentage
df[df['days_employed'] > 0]['days_employed'].count() / df.shape[0] * 100


16.031212036258292

In [21]:
# Find out which 'income_type' values are associated with the problematic data
df[df['days_employed'] > 0]['income_type'].value_counts()


retiree       3429
unemployed       2
Name: income_type, dtype: int64

Looking at the 'days_employed' column, 2 main issues stand out: 1) realistic negative values and 2) unrealistic positive values. The negative values are not necessarily an issue as they may serve to indicate the number of days employed prior to the time of application. They can simply be changed into positive values to prevent calculations errors moving forward. However, there is definitely a need to address the unrealistic positive values as they seem to be mistakes and comprise a substantial 16% of the data. To decipher whether or not there is a pattern to the erroneous data, the values in the 'income_type' column of customers who reported the high positive 'days_employed' values were examined to reveal that the erroneous data only occurs among retirees and unemployed individuals--those who would be expected to have a 'days_employed' value of '0'. Therefore, it seems appopriate to change all the erroneous positive values in the dataset to '0'.

In [22]:
# Address the problematic values in 'days_employed' by changing the large positive values to 0
df.loc[df['days_employed'] > 0, 'days_employed'] = 0


In [23]:
# Check the result to make sure the problematic values were successfully addressed
print(df.head(20))


    children  days_employed  dob_years            education  education_id  \
0          1   -8437.673028         42    bachelor's degree             0   
1          1   -4024.803754         36  secondary education             1   
2          0   -5623.422610         33  secondary education             1   
3          3   -4124.747207         32  secondary education             1   
4          0       0.000000         53  secondary education             1   
5          0    -926.185831         27    bachelor's degree             0   
6          0   -2879.202052         43    bachelor's degree             0   
7          0    -152.779569         50  secondary education             1   
8          2   -6929.865299         35    bachelor's degree             0   
9          0   -2188.756445         41  secondary education             1   
10         2   -4171.483647         36    bachelor's degree             0   
11         0    -792.701887         40  secondary education             1   

To address the problematic positive values in the 'days_employed' column, they were all changed to the value '0' according to the code above. The dataframe was printed to confirm this change was successful.

In [24]:
# Check the 'dob_years' for potentially erroneous values and count the percentages of each age
print(sorted(df['dob_years'].unique()))
df['dob_years'].value_counts() / df.shape[0] * 100


[0, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75]


35    2.868891
40    2.817494
41    2.817494
34    2.789459
38    2.780114
42    2.766097
33    2.696010
39    2.672647
31    2.597888
36    2.583871
29    2.537146
44    2.537146
30    2.504439
48    2.504439
37    2.481077
43    2.382955
50    2.378282
32    2.364265
49    2.359593
28    2.340903
45    2.308195
27    2.289506
52    2.256798
56    2.252126
47    2.242781
54    2.224091
46    2.191384
58    2.154004
57    2.135314
53    2.135314
51    2.083917
55    2.060555
59    2.060555
26    1.897019
60    1.756845
25    1.663396
61    1.649379
62    1.640034
63    1.252219
64    1.228857
24    1.228857
23    1.177460
65    0.906457
66    0.855060
22    0.855060
67    0.780301
21    0.513971
0     0.467246
68    0.462574
69    0.387814
70    0.303710
71    0.271003
20    0.238295
72    0.154191
19    0.065414
73    0.037380
74    0.028035
75    0.004672
Name: dob_years, dtype: float64

Next, the 'dob_years' column was examined to check for any dubious values. An examination of the list of unique values in the column revealed that the only problematic age was 0. All the other values are feasible. Since the number of rows that include 0 in the 'dob_years' column is only 0.5%, it seems reasonable to simply drop these rows from the analysis.

In [25]:
# Drop the rows in the 'dob_years' column where age = 0
df = df[df['dob_years'] > 0]


In [26]:
# Check the unique values in 'dob_years' to ensure the drop was successful
print(sorted(df['dob_years'].unique()))


[19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75]


In the above code, 'df' was modified to drop rows where the 'dob_years' value is 0. The unique 'dob_years' values were printed once more in order to confirm that all the correct rows were dropped.

In [27]:
# Check the values in the 'family_status' column by printing all unique values
print(sorted(df['family_status'].unique()))


['civil partnership', 'divorced', 'married', 'unmarried', 'widow / widower']


Next, unique values found in the 'family_status' column were examined to check for any potentially problematic values. All of the values seem legitimate so nothing will be done to change this column.

In [28]:
# Check the values in the 'gender' column by printing all unique values
print(sorted(df['gender'].unique()))


['F', 'M', 'XNA']


In [29]:
# Check which rows have the value 'XNA' in the 'gender' column
df[df['gender'] == 'XNA']


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,-2358.600502,24,some college,2,civil partnership,1,XNA,business,0,32624.825,buy real estate


Next, the unique values found in the 'gender' column were examined to check for any potential problematic values. The value 'XNA' was the only potentially problematic value. When the dataset was checked to see which rows have the value 'XNA' in the 'gender' column, it was revealed that it appears in only 1 row. Although it may be that the customer represented by this row does not identify as male or female, it would not be helpful in this analysis to make conclusions about non-binary customers if there is only 1 datapoint that can be analyzed. Therefore, it seems appropriate to drop this row.

In [30]:
# Drop the row where the 'gender' column value is 'XNA'
df.drop(df[df['gender'] == 'XNA'].index, inplace = True)


In [31]:
# Check the unique values in 'gender' to ensure the drop was successful
print(sorted(df['gender'].unique()))


['F', 'M']


The row with 'XNA' in the 'gender' column was dropped and the unique values remaining in the column were printed in order to confirm that the drop was successful.

In [32]:
# Check the values in the 'income_type' column by printing all unique values
print(sorted(df['income_type'].unique()))

['business', 'civil servant', 'employee', 'entrepreneur', 'paternity / maternity leave', 'retiree', 'student', 'unemployed']


Next, the values in the 'income_type' column were examined to check for problematic values. As all of the values seemed legitimate, so no changes were made.

In [33]:
# Check the number of duplicates in 'df'
df.duplicated().sum()


71

In [34]:
# Find the percentage of duplicated values in df
df.duplicated().sum() / df.shape[0] * 100


0.33331768461574574

Finally, the dataset was checked for duplicate values. 71 duplicate values were identified, which comprise a rather small 0.3% of the data. Therefore, the duplicates will be dropped from the analysis to ensure that duplicate rows do not skew the results.

In [35]:
# Drop the duplicate values from df
df = df.drop_duplicates().reset_index(drop = True)


In [36]:
# Check if all the duplicates were dropped
df.duplicated().sum()


0

The 71 duplicate rows were dropped. The drop was confirmed by checking the number of remaining duplicate values which is 0.

In [37]:
# Check the size of the remaining dataset after the first set of manipulations
df.shape[0]


21230

In [38]:
# Calculate the percentage of data that was dropped from the dataset
(original_shape[0] - df.shape[0]) / original_shape[0] * 100


1.3704994192799071

Having gone through the important variables in the dataset and addressing all the problematic values, the current dataset contains 21,230 rows compared to the original 21,525 which indicates that 1.37% of the original data was dropped through the data cleaning process. The dropped rows were all deemed problematic and included customers that reported -1 or 20 children, age of 0 years, and 'XNA' gender. Duplicate rows were also dropped. Since 1.37% is not a significant portion of the data, the remaining dataset should still be valid to use in the remainder of the analysis.

# Working with missing values

Prior to working with the missing values, dictionaries will be created for the 'education_id' and 'education' columns, as well as the 'family_status_id' and 'family_status' columns just in case they may be needed to help speed up the process and make it easier to call values.

In [39]:
# Create a dictionary for 'education_id' and 'education'
education_dict = df[['education_id', 'education']].drop_duplicates().reset_index(drop=True)
print(education_dict)


   education_id            education
0             0    bachelor's degree
1             1  secondary education
2             2         some college
3             3    primary education
4             4      graduate degree


In [40]:
# Create a dictionary for 'family_status_id' and 'family_status'
family_status_dict = df[['family_status_id', 'family_status']].drop_duplicates().reset_index(drop=True)
print(family_status_dict)


   family_status_id      family_status
0                 0            married
1                 1  civil partnership
2                 2    widow / widower
3                 3           divorced
4                 4          unmarried


### Restoring missing values in `total_income`

The two columns that have missing values that still need to be addressed are the 'total_income' and 'days_employed' column. In this section, the missing values in the 'total_income' column will be fixed by examining the relationship between income and some other variables of relevance in the dataset, namely age, education, and income type. Then the median or mean values of 'total_income' calculated from the non-missing data based on these variables of relevance can then be used to fill in the missing values in rows with similar characteristics.

In [41]:
# Write a function that calculates the age category
def age_category(age):
    if age <= 19:
        return '10-19'
    elif age <= 29:
        return '20-29'
    elif age <= 39:
        return '30-39'
    elif age <= 49:
        return '40-49'
    elif age <= 59:
        return '50-59'
    elif age <= 69:
        return '60-69'
    elif age <= 79:
        return '70-79'
    else:
        return "unknown"
    

In [42]:
# Test if the function works with sample age values
print(age_category(19))
print(age_category(75))
print(age_category(100))


10-19
70-79
unknown


Prior to proceeding in the restoration of missing values in the 'total_income' column, the 'age' column will be categorized into age groupings for ease of analysis. Above is a function that calculates the age category depending on the age value that is inputted. It categorizes the inputted age in 10 year increments from age 10-79 as the lowest and highest age values in the dataset are 19 and 75 respectively. A few tests were run to ensure that the function is working properly.

In [43]:
# Create a new 'age_category' column by applying the 'age_category' function on the 'dob_years' column
df['age_category'] = df['dob_years'].apply(age_category)


In [44]:
# Check the values in the 'age_category' column alongside the 'dob_years' column
print(df[['dob_years', 'age_category']].head(20))


    dob_years age_category
0          42        40-49
1          36        30-39
2          33        30-39
3          32        30-39
4          53        50-59
5          27        20-29
6          43        40-49
7          50        50-59
8          35        30-39
9          41        40-49
10         36        30-39
11         40        40-49
12         65        60-69
13         54        50-59
14         56        50-59
15         26        20-29
16         35        30-39
17         33        30-39
18         53        50-59
19         48        40-49


Using the age_category function, a new column called 'age_category' was created in the dataset that categorizes the ages to help make it easier to perform comparisons later in the analysis. The 'dob_years' and the new 'age_category' columns were printed side by side to ensure that the new column was successfully and accurately generated.

Now there is a need to begin looking at how age, education, and income type are related to total income to determine whether mean or median values should be used to replace the missing values in the 'total_income' column and which variables should ultimately be considered to make that calculation.

In [45]:
# Create a table without missing values and print a few of its rows to make sure it was accurately generated
new_df = df.dropna()
print(new_df.head(20))


    children  days_employed  dob_years            education  education_id  \
0          1   -8437.673028         42    bachelor's degree             0   
1          1   -4024.803754         36  secondary education             1   
2          0   -5623.422610         33  secondary education             1   
3          3   -4124.747207         32  secondary education             1   
4          0       0.000000         53  secondary education             1   
5          0    -926.185831         27    bachelor's degree             0   
6          0   -2879.202052         43    bachelor's degree             0   
7          0    -152.779569         50  secondary education             1   
8          2   -6929.865299         35    bachelor's degree             0   
9          0   -2188.756445         41  secondary education             1   
10         2   -4171.483647         36    bachelor's degree             0   
11         0    -792.701887         40  secondary education             1   

Above, a new table that excludes all missing values was created by dropping all the missing values. The drop was successful based on the printed output. The new dataset, 'new_df' will be utilized to examine the mean and median total income values based on age, education, and income type.

In [46]:
# Look at the mean values for income based on age
print(new_df.groupby('age_category')['total_income'].mean())


age_category
10-19    16993.942462
20-29    25583.136962
30-39    28314.525654
40-49    28575.427654
50-59    25807.707523
60-69    23236.985508
70-79    20125.658331
Name: total_income, dtype: float64


In [47]:
# Look at the median values for income based on age
print(new_df.groupby('age_category')['total_income'].median())


age_category
10-19    14934.9010
20-29    22804.3590
30-39    24699.5815
40-49    24772.3820
50-59    22195.8470
60-69    19811.3260
70-79    18751.3240
Name: total_income, dtype: float64


In [48]:
# Look at the mean values for income based on education
print(new_df.groupby('education')['total_income'].mean())


education
bachelor's degree      33197.258790
graduate degree        27960.024667
primary education      21144.882211
secondary education    24594.390815
some college           29028.844227
Name: total_income, dtype: float64


In [49]:
# Look at the median values for income based on education
print(new_df.groupby('education')['total_income'].median())


education
bachelor's degree      28086.5425
graduate degree        25161.5835
primary education      18741.9760
secondary education    21832.2410
some college           25618.4640
Name: total_income, dtype: float64


In [50]:
# Look at the mean values for income based on income type
print(new_df.groupby('income_type')['total_income'].mean())


income_type
business                       32424.420789
civil servant                  27336.442546
employee                       25822.872585
entrepreneur                   79866.103000
paternity / maternity leave     8612.661000
retiree                        21950.722935
student                        15712.260000
unemployed                     21014.360500
Name: total_income, dtype: float64


In [51]:
# Look at the median values for income based on type
print(new_df.groupby('income_type')['total_income'].median())


income_type
business                       27594.6410
civil servant                  24076.1150
employee                       22815.1035
entrepreneur                   79866.1030
paternity / maternity leave     8612.6610
retiree                        18959.6260
student                        15712.2600
unemployed                     21014.3605
Name: total_income, dtype: float64


Based on the above results, it does appear that all 3 variables of interest--age, education, and income type--do affect the mean and median total income values in a fairly significant manner:

1) Age shows an upwards trend in income that peaks in the 40-49 age category and then gradually diminishes with each older age category.

2) Education shows that those with college degrees or at least some college education have higher income than those without.

3) Income type reveals that those who are currently employed in some capacity (business owners, civil servants, employees, and entrepreneurs) earn more than those who are not (those on paternity/maternity leave, retirees, students, and the unemployed).

Although the mean and median values within each variable of interest are different, the trends remain the same. The main difference is that the mean values are pretty consistently higher than the median values which indicates that there may be some high income outliers that are pulling up the mean values in a way that may make it an overestimation of the actual average income. Therefore, the median will be utilized to calculate and fill in the missing values found in the 'total_income' column.

Given these results, the median value associated with the combination of the 'age_category', 'education', and 'income_type' variables will be utilized to fill in the missing values in the 'total_income' column.

In [52]:
# Create a new column containing a list of string values from columns of interest for each row
df['filler'] = df[['age_category', 'education', 'income_type', 'total_income', 'days_employed']].values.tolist()


To start, a new column called 'filler' was created in df that contains a list of string values from the 'age_category', 'education', 'income_type', and 'total_income' columns which are needed as inputs for the function that is written below.

In [53]:
# Write a function that we will use for filling in missing values
def fill_total_income(filler):
    if pd.isna(filler[3]):
        return new_df[(new_df['age_category'] == filler[0]) & (new_df['education'] == filler[1]) & (new_df['income_type'] == filler[2])]['total_income'].median()
    else:
        return filler[3]


The function 'fill_total_income' is designed to take in the 'filler' list and determine which rows in the 'total_income' column are missing. When a missing value is located, it returns the median value of total income associated with the particular combination of the variables of interest to fill in the missing value. However, if no missing value is located, it returns the value found in the column and does not attempt to change it.

In [54]:
# Print the median values of 'total_income' grouped by the variables of interest
print(new_df.groupby(['age_category', 'education', 'income_type'])['total_income'].median())


age_category  education            income_type  
10-19         secondary education  business         16588.2370
                                   employee         14934.9010
              some college         business         26598.4335
                                   civil servant    12125.9860
                                   employee         13752.8305
                                                       ...    
70-79         secondary education  business         24259.6870
                                   civil servant    19530.6775
                                   employee         24660.9010
                                   retiree          17543.2480
              some college         retiree          14479.1930
Name: total_income, Length: 97, dtype: float64


In [55]:
# Check if the function works based on the results of the grouping above
fill_total_income(['10-19', 'secondary education', 'business', float('NaN')])


16588.237

In [56]:
# Check if the function works in the case that it encounters a non-missing value in the 'total_income' column
fill_total_income(['10-19', 'secondary education', 'business', 10])

10

The 'fill_total_income' function was tested by first printing out the median values of total income based on the combination of the 3 variables of interest. Then, a sample string based on the first row of the groupby output was utilized to check if the function would output the correct value--16588.237. After checking that portion of the function, the case where the function encounters a non-missing value in the 'total_income' column was tested to ensure that it would return the original value, which it did.

In [57]:
# Apply the 'fill_total_income' function to every row in 'total_income' and print the first 15 rows
df['total_income'] = df['filler'].apply(fill_total_income)
df.head(15)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category,filler
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,40-49,"[40-49, bachelor's degree, employee, 40620.102..."
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,30-39,"[30-39, secondary education, employee, 17932.8..."
2,0,-5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,30-39,"[30-39, secondary education, employee, 23341.7..."
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,30-39,"[30-39, secondary education, employee, 42820.5..."
4,0,0.000000,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,50-59,"[50-59, secondary education, retiree, 25378.57..."
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,20-29,"[20-29, bachelor's degree, business, 40922.17,..."
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,40-49,"[40-49, bachelor's degree, business, 38484.156..."
7,0,-152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education,50-59,"[50-59, secondary education, employee, 21731.8..."
8,2,-6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,30-39,"[30-39, bachelor's degree, employee, 15337.093..."
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,40-49,"[40-49, secondary education, employee, 23108.1..."


Since the function appears to be functioning correctly, it was applied to each row in the 'total_income' column of df by taking in each list from the 'filler' column. The first 15 rows of the new list was printed to ensure that the application was successful. Based on row 12 where the value in the 'total_income' column was originally NaN but is now filled with the appropriate median value, we can conclude that the function was successfully applied.

In [58]:
# Check if there are any remaining missing values in 'total_income'
df.isna().sum()


children               0
days_employed       2081
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           3
purpose                0
age_category           0
filler                 0
dtype: int64

In [59]:
# Check which specific rows still have a missing value in the 'total_income' column
print(df[df['total_income'].isna()])


      children  days_employed  dob_years          education  education_id  \
1288         1            NaN         70  primary education             3   
5880         0            NaN         58  bachelor's degree             0   
8055         0            NaN         64  primary education             3   

          family_status  family_status_id gender    income_type  debt  \
1288  civil partnership                 1      F       employee     0   
5880            married                 0      M   entrepreneur     0   
8055  civil partnership                 1      F  civil servant     0   

      total_income                                   purpose age_category  \
1288           NaN  transactions with commercial real estate        70-79   
5880           NaN               buy residential real estate        50-59   
8055           NaN                         to have a wedding        60-69   

                                                 filler  
1288     [70-79, primary educat

However, although the function was successfully applied to df, it appears that there are still 3 missing values in the 'total_income' column remaining, likely because there was not enough data to calculate the median value based on the given parameters. Therefore, it left the value in the column as NaN. 

In [60]:
# Write another function that we will use for filling in the 3 remaining missing values that doesn't utilize 'income_type'
def fill_total_income_2(filler):
    if pd.isna(filler[3]):
        return new_df[(new_df['age_category'] == filler[0]) & (new_df['education'] == filler[1])]['total_income'].median()
    else:
        return filler[3]
    

In order to deal with the 3 remaining missing values, a new function called 'fill_total_income_2' was created. It functions almost identically to the original 'fill_total_income' function except that it does not utilize 'income_type' to calculate the median value, just the values from the 'age_category' and 'education' columns since 'income_type' takes on more potential values which makes it more likely to lead to not having enough data to calculate the median values.

In [61]:
# Apply the 'fill_total_income_2' function to df to fill in the 3 missing values in 'total_income'
df['total_income'] = df['filler'].apply(fill_total_income_2)


The 'fill_total_income_2' function was applied to the 'total_income' column in df to fill in the remaining missing values.

In [62]:
# Check the number of missing values remaining in the 'total_income' column
df.isna().sum()


children               0
days_employed       2081
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
age_category           0
filler                 0
dtype: int64

Finally, df was checked for missing values after applying the function, revealing that none remain. We can therefore conclude that all the missing values in the 'total_income' column were successfully dealt with, allowing us to proceed in the analysis.

###  Restoring values in `days_employed`

Now, the only column with missing values that still need to be addressed is the 'days_employed' column in df. In this section, the missing values in the 'days_employed' column will be fixed by examining the relationship between days of employment and the same variables of relevance employed in restoring the missing values in the 'total_income' column--age, education, and income type. Then the median or mean values of 'days_employed' calculated from the non-missing data based on these variables of relevance which can then be used to fill in the missing values in rows with similar characteristics.

First, there is a need to examine how age, education, and income type are related to the number of days employed to determine whether mean or median values should be used to replace the missing values in the 'days_employed' column and which variables should ultimately be considered to make that calculation.

In [63]:
# Change all the negative values in the 'days_employed' column to positive to prevent calculation errors in df
df['days_employed'] = df['days_employed'].abs()
df.head(15)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category,filler
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,40-49,"[40-49, bachelor's degree, employee, 40620.102..."
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,30-39,"[30-39, secondary education, employee, 17932.8..."
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,30-39,"[30-39, secondary education, employee, 23341.7..."
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,30-39,"[30-39, secondary education, employee, 42820.5..."
4,0,0.000000,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,50-59,"[50-59, secondary education, retiree, 25378.57..."
5,0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,20-29,"[20-29, bachelor's degree, business, 40922.17,..."
6,0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,40-49,"[40-49, bachelor's degree, business, 38484.156..."
7,0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education,50-59,"[50-59, secondary education, employee, 21731.8..."
8,2,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,30-39,"[30-39, bachelor's degree, employee, 15337.093..."
9,0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,40-49,"[40-49, secondary education, employee, 23108.1..."


In [64]:
# Change all the negative values in the 'days_employed' column to positive to prevent calculation errors in new_df
new_df['days_employed'] = new_df.loc[:, 'days_employed'].abs()
new_df.head(15)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,40-49
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,30-39
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,30-39
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,30-39
4,0,0.000000,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,50-59
5,0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,20-29
6,0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,40-49
7,0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education,50-59
8,2,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,30-39
9,0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,40-49


Prior to proceeding to examine the relationship between 'days_employed' and the variables of interest, the negative values in the 'days_employed' column were changed to positive values in df and new_df to prevent calculation errors in the remainder of the analysis.

In [65]:
# Distribution of 'days_employed' means based on the 'age_category' column
print(new_df.groupby('age_category')['days_employed'].mean())


age_category
10-19     633.678086
20-29    1208.812282
30-39    2014.334698
40-49    2662.529357
50-59    2092.263603
60-69     862.366723
70-79     528.351115
Name: days_employed, dtype: float64


In [66]:
# Distribution of 'days_employed' medians based on the 'age_category' column
print(new_df.groupby('age_category')['days_employed'].median())


age_category
10-19     724.492610
20-29     998.961907
30-39    1580.160806
40-49    1946.616479
50-59     879.731714
60-69       0.000000
70-79       0.000000
Name: days_employed, dtype: float64


In [67]:
# Distribution of 'days_employed' means based on the 'education' column
print(new_df.groupby('education')['days_employed'].mean())


education
bachelor's degree      2026.919285
graduate degree        2346.791024
primary education      1249.168800
secondary education    1938.468647
some college           1476.702678
Name: days_employed, dtype: float64


In [68]:
# Distribution of 'days_employed' medians based on the 'education' column
print(new_df.groupby('education')['days_employed'].median())


education
bachelor's degree      1342.118520
graduate degree        1380.316041
primary education       551.062561
secondary education    1186.705918
some college           1048.028924
Name: days_employed, dtype: float64


In [69]:
# Distribution of 'days_employed' means based on the 'income_type' column
print(new_df.groupby('income_type')['days_employed'].mean())


income_type
business                       2119.512866
civil servant                  3392.119263
employee                       2325.740892
entrepreneur                    520.848083
paternity / maternity leave    3296.759962
retiree                           0.000000
student                         578.751554
unemployed                        0.000000
Name: days_employed, dtype: float64


In [70]:
# Distribution of 'days_employed' means based on the 'income_type' column
print(new_df.groupby('income_type')['days_employed'].median())


income_type
business                       1555.993659
civil servant                  2672.903939
employee                       1573.791064
entrepreneur                    520.848083
paternity / maternity leave    3296.759962
retiree                           0.000000
student                         578.751554
unemployed                        0.000000
Name: days_employed, dtype: float64


Based on the above results, it does appear that all 3 variables of interest--age, education, and income type--do affect the mean and median days employed values in a fairly significant manner:

1) Age shows an upwards trend in number of days employed that peaks in the 40-49 age category and then diminishes with each older age category.

2) Education shows slightly different trends based on mean versus median. The mean days employed values tend to be highest among college graduates and those with up to secondary education. Those with some college education and primary education had the lowest mean days employed values. However, the median days employed values for all the education categories were relatively similar for all but primary education which was significantly lower.

3) Income type showed greater variety between the various categories with similar trends among business owners, civil servants, and employees for the mean and median values of days employed and the exact same values for the remaining categories.

In the cases where the mean and median values seem to differ greatly, it appears that there may be some significant outliers that are causing the days employed values to be higher and are thus potentially overestimating the actual value. Therefore, it seems most appropriate to use the median values.

In conclusion, the median value associated with the combination of the 'age_category', 'education', and 'income_type' variables will be utilized to fill in the missing values in the 'days_employed' column.

In [71]:
# Write a function that we will use for filling in missing values with medians calculated based on the variables of interest
def fill_days_employed(filler):
    if pd.isna(filler[4]):
        return new_df[(new_df['age_category'] == filler[0]) & (new_df['education'] == filler[1]) & (new_df['income_type'] == filler[2])]['days_employed'].median()
    else:
        return filler[4]


The function 'fill_days_employed' is designed to take in the 'filler' list and determine which rows in the 'days_employed' column are missing. When a missing value is located, it returns the median value of days employed associated with the particular combination of the variables of interest to fill in the missing value. However, if no missing value is located, it returns the value found in the column and does not attempt to change it.

In [72]:
# Print the median values of 'days_employed' grouped by the variables of interest
print(new_df.groupby(['age_category', 'education', 'income_type'])['days_employed'].median())


age_category  education            income_type  
10-19         secondary education  business          322.024011
                                   employee          793.358581
              some college         business          822.097347
                                   civil servant     509.969922
                                   employee          841.126105
                                                       ...     
70-79         secondary education  business         2598.981129
                                   civil servant    1204.922098
                                   employee         1504.924191
                                   retiree             0.000000
              some college         retiree             0.000000
Name: days_employed, Length: 97, dtype: float64


In [73]:
# Check if the function works based on the results of the grouping above
fill_days_employed(['10-19', 'secondary education', 'business', 0, float('NaN')])


322.0240107163265

In [74]:
# Check if the function works in the case that it encounters a non-missing value in the 'days_employed' column
fill_days_employed(['10-19', 'secondary education', 'business', 0, 10])

10

The 'fill_days_employed' function was tested by first printing out the median values of days employed based on the combination of the 3 variables of interest. Then, a sample string based on the first row of the groupby output was utilized to check if the function would output the correct value--322.024011. After checking that portion of the function, the case where the function encounters a non-missing value in the 'days_employed' column was tested to ensure that it would return the original value, which it did.

In [75]:
# Apply the 'fill_days_employed' function to every row in 'days_employed'
df['days_employed'] = df['filler'].apply(fill_days_employed)
df.head(15)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category,filler
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,40-49,"[40-49, bachelor's degree, employee, 40620.102..."
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,30-39,"[30-39, secondary education, employee, 17932.8..."
2,0,-5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,30-39,"[30-39, secondary education, employee, 23341.7..."
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,30-39,"[30-39, secondary education, employee, 42820.5..."
4,0,0.000000,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,50-59,"[50-59, secondary education, retiree, 25378.57..."
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,20-29,"[20-29, bachelor's degree, business, 40922.17,..."
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,40-49,"[40-49, bachelor's degree, business, 38484.156..."
7,0,-152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education,50-59,"[50-59, secondary education, employee, 21731.8..."
8,2,-6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,30-39,"[30-39, bachelor's degree, employee, 15337.093..."
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,40-49,"[40-49, secondary education, employee, 23108.1..."


Since the function appears to be functioning correctly, it was applied to each row in the 'days_employed' column of df by taking in each list from the 'filler' column. The first 15 rows of the new dataset was printed to ensure that the application was successful. Based on row 12 where the value in the 'days_employed' column was originally NaN but is now filled with the appropriate median value, we can conclude that the function was successfully applied.

In [76]:
# Check if there are any remaining missing values in 'days_employed'
df.isna().sum()


children            0
days_employed       3
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
age_category        0
filler              0
dtype: int64

In [77]:
# Check which specific rows still have a missing value in the 'days_employed' column
print(df[df['days_employed'].isna()])


      children  days_employed  dob_years          education  education_id  \
1288         1            NaN         70  primary education             3   
5880         0            NaN         58  bachelor's degree             0   
8055         0            NaN         64  primary education             3   

          family_status  family_status_id gender    income_type  debt  \
1288  civil partnership                 1      F       employee     0   
5880            married                 0      M   entrepreneur     0   
8055  civil partnership                 1      F  civil servant     0   

      total_income                                   purpose age_category  \
1288    15013.5050  transactions with commercial real estate        70-79   
5880    28152.1765               buy residential real estate        50-59   
8055    17657.4995                         to have a wedding        60-69   

                                                 filler  
1288     [70-79, primary educat

However, although the function was successfully applied to df, it appears that there are still 3 missing values in the 'days_employed' column remaining, likely because there was not enough data to calculate the median value based on the given parameters. Therefore, it left the value in the column as NaN.

In [78]:
# Write another function that we will use for filling in the 3 remaining missing values that doesn't utilize 'income_type'
def fill_days_employed_2(filler):
    if pd.isna(filler[4]):
        return new_df[(new_df['age_category'] == filler[0]) & (new_df['education'] == filler[1])]['days_employed'].median()
    else:
        return filler[4]
    

In order to deal with the 3 remaining missing values, a new function called 'fill_days_employed_2' was created. It functions almost identically to the original 'fill_days_employed' function except that it does not utilize 'income_type' to calculate the median value, just the values from the 'age_category' and 'education' columns since 'income_type' takes on more potential values which makes it more likely to lead to not having enough data to calculate the median values.

In [79]:
# Apply the 'fill_total_income_2' function to df to fill in the 3 missing values in 'days_employed'
df['days_employed'] = df['filler'].apply(fill_days_employed_2)


The 'fill_days_employed_2' function was applied to the 'days_employed' column in df to fill in the remaining missing values.

In [80]:
# Check the columns in df and the number of missing values remaining in the 'days_employed' column
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21230 entries, 0 to 21229
Data columns (total 14 columns):
children            21230 non-null int64
days_employed       21230 non-null float64
dob_years           21230 non-null int64
education           21230 non-null object
education_id        21230 non-null int64
family_status       21230 non-null object
family_status_id    21230 non-null int64
gender              21230 non-null object
income_type         21230 non-null object
debt                21230 non-null int64
total_income        21230 non-null float64
purpose             21230 non-null object
age_category        21230 non-null object
filler              21230 non-null object
dtypes: float64(2), int64(5), object(7)
memory usage: 2.3+ MB


Finally, df was checked for missing values after applying the function, revealing that none remain. We can therefore conclude that all the missing values in the 'days_employed' column were successfully dealt with, allowing us to proceed in the analysis.

In [81]:
# Drop the 'filler' column as it was only created to perform the function and is no longer needed in the dataset
df = df.drop('filler', 1)


In [82]:
# Confirm that the 'filler' column was dropped
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21230 entries, 0 to 21229
Data columns (total 13 columns):
children            21230 non-null int64
days_employed       21230 non-null float64
dob_years           21230 non-null int64
education           21230 non-null object
education_id        21230 non-null int64
family_status       21230 non-null object
family_status_id    21230 non-null int64
gender              21230 non-null object
income_type         21230 non-null object
debt                21230 non-null int64
total_income        21230 non-null float64
purpose             21230 non-null object
age_category        21230 non-null object
dtypes: float64(2), int64(5), object(6)
memory usage: 2.1+ MB


Prior to proceeding to the next step, the 'filler' column that was used to perform the functions used to fill in the missing values of the 'total_income' and 'days_employed' columns was dropped since it is no longer needed in the dataset. The drop was confirmed using the df.info() method.

## Categorization of data

Looking forward at Section 5 where various hypotheses related to the task at hand will be checked, there are 2 columns that must be analyzed but are currently uncategorized and thus difficult to draw conclusions from: the 'purpose' column which has individualized descriptions of the purpose that each individual wrote out without a standardized format, and the 'total_income' column which features a wide variety of income values without any income brackets that would help to conceptualize low versus high income in a concrete manner. Therefore in this section, these two columns will be categorized into discrete, standard categories.

The 'purpose' column will be categorized first:

In [83]:
# Print the values in the 'purpose' column
print(df['purpose'])


0          purchase of the house
1                   car purchase
2          purchase of the house
3        supplementary education
4              to have a wedding
                  ...           
21225       housing transactions
21226          purchase of a car
21227                   property
21228          buying my own car
21229               to buy a car
Name: purpose, Length: 21230, dtype: object


Looking at some of the values in the 'purpose' column printed above, it appears that there are many different stated purposes but they largely fall into just a few overarching categories.

In [84]:
# Check the unique values in 'purpose'
print(df['purpose'].unique())


['purchase of the house' 'car purchase' 'supplementary education'
 'to have a wedding' 'housing transactions' 'education' 'having a wedding'
 'purchase of the house for my family' 'buy real estate'
 'buy commercial real estate' 'buy residential real estate'
 'construction of own property' 'property' 'building a property'
 'buying a second-hand car' 'buying my own car'
 'transactions with commercial real estate' 'building a real estate'
 'housing' 'transactions with my real estate' 'cars' 'to become educated'
 'second-hand car purchase' 'getting an education' 'car'
 'wedding ceremony' 'to get a supplementary education'
 'purchase of my own house' 'real estate transactions'
 'getting higher education' 'to own a car' 'purchase of a car'
 'profile education' 'university education'
 'buying property for renting out' 'to buy a car' 'housing renovation'
 'going to university']


In [85]:
# Create a list of all the lemmas found in 'purpose' using a 'for' loop
lemmas_list = []

for purpose in df['purpose'].unique():
    words = nltk.word_tokenize(purpose)
    lemmas = [wordnet_lemma.lemmatize(w, pos = 'n') for w in words]
    for i in lemmas:
        lemmas_list.append(i)


In [86]:
# Check lemmas_list
lemmas_list


['purchase',
 'of',
 'the',
 'house',
 'car',
 'purchase',
 'supplementary',
 'education',
 'to',
 'have',
 'a',
 'wedding',
 'housing',
 'transaction',
 'education',
 'having',
 'a',
 'wedding',
 'purchase',
 'of',
 'the',
 'house',
 'for',
 'my',
 'family',
 'buy',
 'real',
 'estate',
 'buy',
 'commercial',
 'real',
 'estate',
 'buy',
 'residential',
 'real',
 'estate',
 'construction',
 'of',
 'own',
 'property',
 'property',
 'building',
 'a',
 'property',
 'buying',
 'a',
 'second-hand',
 'car',
 'buying',
 'my',
 'own',
 'car',
 'transaction',
 'with',
 'commercial',
 'real',
 'estate',
 'building',
 'a',
 'real',
 'estate',
 'housing',
 'transaction',
 'with',
 'my',
 'real',
 'estate',
 'car',
 'to',
 'become',
 'educated',
 'second-hand',
 'car',
 'purchase',
 'getting',
 'an',
 'education',
 'car',
 'wedding',
 'ceremony',
 'to',
 'get',
 'a',
 'supplementary',
 'education',
 'purchase',
 'of',
 'my',
 'own',
 'house',
 'real',
 'estate',
 'transaction',
 'getting',
 'higher'

The above output of the lemmatization of the strings within the 'purpose' column confirms the prior observation. In order to put these strings into neat categories, the list of lemmas was analyzed to develop 4 distinct categories that accommodates all the unique values above: 1) real estate, 2) car, 3) education, and 4) wedding. Then, all unique keywords related to each category were identified to ensure that the category could capture all related statements. The following is a list of the categories and their associated keywords:

1) Real Estate: 'house', 'housing', 'property', 'real', 'estate', 'construction'

2) Car: 'car', 'cars'

3) Education: 'education', 'educated', 'university'

4) Wedding: 'wedding'


In [87]:
# Create categories based on the lemmas that were returned from the strings in 'purpose'
real_estate = ['house', 'housing', 'property', 'real', 'estate', 'construction']
car = ['car', 'cars']
education = ['education', 'educated', 'university']
wedding = ['wedding']


In [88]:
# Write a function to categorize the values in 'purpose' into the above categories
def categorize_purpose(string):
    
    words = nltk.word_tokenize(string)
    lemmas = [wordnet_lemma.lemmatize(w, pos = 'n') for w in words]
    
    if any(word in lemmas for word in real_estate):
        return 'real estate'
    elif any(word in lemmas for word in car):
        return 'car'
    elif any(word in lemmas for word in education):
        return 'education'
    elif any(word in lemmas for word in wedding):
        return 'wedding'
    else:
        return 'unknown'


The function 'categorize_purpose' was written to take in values from the 'purpose' column and output 1 of the 4 defined categories or 'unknown' category depending on the keywords that are present in the 'purpose' column string value.

In [89]:
# Check to see if function works for all 4 categories of 'purpose' and the 'unknown' case
categorize_purpose('purchase of the house')


'real estate'

In [90]:
categorize_purpose('car purchase')

'car'

In [91]:
categorize_purpose('supplementary education')

'education'

In [92]:
categorize_purpose('to have a wedding')

'wedding'

In [93]:
categorize_purpose('unknown')

'unknown'

The 'categorize_purpose' function was tested above using sample strings from each of the possible categories. The results show that it is functioning properly.

In [94]:
# Create a new column 'purpose_category' by applying the 'categorize_purpose' function to the 'purpose' column
df['purpose_category'] = df['purpose'].apply(categorize_purpose)


A new column called 'purpose_category' was created by applying the 'categorize_purpose' function on the 'purpose' column.

In [95]:
# Check to see if the column was successfully created
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21230 entries, 0 to 21229
Data columns (total 14 columns):
children            21230 non-null int64
days_employed       21230 non-null float64
dob_years           21230 non-null int64
education           21230 non-null object
education_id        21230 non-null int64
family_status       21230 non-null object
family_status_id    21230 non-null int64
gender              21230 non-null object
income_type         21230 non-null object
debt                21230 non-null int64
total_income        21230 non-null float64
purpose             21230 non-null object
age_category        21230 non-null object
purpose_category    21230 non-null object
dtypes: float64(2), int64(5), object(7)
memory usage: 2.3+ MB


In [96]:
# Count the values in the 'purpose_category' column
df['purpose_category'].value_counts()


real estate    10703
car             4258
education       3970
wedding         2299
Name: purpose_category, dtype: int64

df.info() was called to confirm that the column was successfully created and the value_counts() method was called to check how many rows there are for each category.

Next, the 'total_income' column will be categorized:

In [97]:
# Look through some of the values in 'total_income'
print(df['total_income'])


0        40620.102
1        17932.802
2        23341.752
3        42820.568
4        25378.572
           ...    
21225    35966.698
21226    24959.969
21227    14347.610
21228    39054.888
21229    13127.587
Name: total_income, Length: 21230, dtype: float64


The values in the 'total_income" column were examined, showing a fairly wide range of values.

In [98]:
# Getting summary statistics for the 'total_income' column
df['total_income'].describe()


count     21230.000000
mean      26477.297629
std       15749.250695
min        3306.762000
25%       17197.731250
50%       23001.785000
75%       31327.351000
max      362496.645000
Name: total_income, dtype: float64

Summary statistics were calculated for the 'total_income' column which delineated important numerical cut-offs including the min, 25th percentile, 50th percentile, 75th percentile, and the max. These values seem to be appropriate markers to create income bracket categories.

In [99]:
# Create a function for categorizing 'total_income' values into categories based on quartiles
def income_cat(value):
    if value <= 17197.731250:
        return '1'
    elif value <= 23001.785000:
        return '2'
    elif value <= 31327.351000:
        return '3'
    elif value <= 362496.645000:
        return '4'
    else:
        return "unknown"


The function 'income_cat' was created to take in an income value and place it into an income category from 1-4 which correspond to 1-4 quartile ranges (1 representing the lowest income quartile and 4 representing the highest income quartile). The min/max and percentile cut-off values were utilized from the outcome of the describe() method above to create the income categories.

In [100]:
# Check to see if the 'income_cat' function works using sample income values
income_cat(17197.731250)


'1'

In [101]:
income_cat(20000)

'2'

In [102]:
income_cat(400000)

'unknown'

Sample income values were used to test the 'income_cat' function above and showed that it is indeed functioning properly.

In [103]:
# Create a new column 'income_category' by applying the 'income_cat' function to the 'total_income' column
df['income_category'] = df['total_income'].apply(income_cat)


A new column called 'income_category' was created by applying the 'income_cat' function to the values in the 'total_income' column.

In [104]:
# Check to see if the 'income_category' column was successfully created
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21230 entries, 0 to 21229
Data columns (total 15 columns):
children            21230 non-null int64
days_employed       21230 non-null float64
dob_years           21230 non-null int64
education           21230 non-null object
education_id        21230 non-null int64
family_status       21230 non-null object
family_status_id    21230 non-null int64
gender              21230 non-null object
income_type         21230 non-null object
debt                21230 non-null int64
total_income        21230 non-null float64
purpose             21230 non-null object
age_category        21230 non-null object
purpose_category    21230 non-null object
income_category     21230 non-null object
dtypes: float64(2), int64(5), object(8)
memory usage: 2.4+ MB


In [105]:
# Count the values in the 'purpose_category' column
df['income_category'].value_counts()


2    5631
1    5308
4    5308
3    4983
Name: income_category, dtype: int64

df.info() was called to confirm that the column was successfully created and the value_counts() method was called to check how many rows there are for each category.

## Checking the Hypotheses


**Is there a correlation between having children and paying back on time?**

In [106]:
# Check the relationship between the values in 'children' and 'debt' using a pivot table aggregating on the sum
children_debt_pt_sum = pd.pivot_table(data = df, index = ['children'], values = ['debt'], aggfunc = 'sum')
print(children_debt_pt_sum)


          debt
children      
0         1058
1          441
2          194
3           27
4            4
5            0


In [107]:
# Check the relationship between the values in 'children' and 'debt' using a pivot table aggregating on the count
children_debt_pt_count = pd.pivot_table(data = df, index = ['children'], values = ['debt'], aggfunc = 'count')
print(children_debt_pt_count)


           debt
children       
0         14021
1          4792
2          2039
3           328
4            41
5             9


Pivot tables were generated to display the relationship between the values in 'children' and 'debt'. The first table aggregates on the sum to calculate the number of rows in each category of 'children' that defaulted on a loan while the second table aggregates on the count to calculate the number of total rows in each category of 'children'.

In [108]:
# Calculate the default-rate based on the number of children
children_debt_pt_sum / children_debt_pt_count * 100


,debt
children,
0,7.545824
1,9.202838
2,9.514468
3,8.231707
4,9.756098
5,0.000000


The default-rate based on the number of children was calculated by dividing the sum pivot table by the count pivot table and multiplying the result by 100 to get the percentage of individuals within each category of 'children' that has ever defaulted on a loan.

**Conclusion**

Based on the results of the calculations, it appears that individuals with no children have lower default rates than those with 1 or more children. Although the default rate for 5 children is technically 0, there are not enough data points in that category of children to draw meaningful conclusions. Therefore, overall it does appear that individuals with children have higher default rates than those without children. This trend may be due to the fact that children represent an additional expense that can place families under financial stress with little funds available to put aside for repayment of loans.

**Is there a correlation between family status and paying back on time?**

In [109]:
# Check the relationship between the values in 'family_status' and 'debt' using a pivot table aggregating on the sum
family_debt_pt_sum = pd.pivot_table(data = df, index = ['family_status'], values = ['debt'], aggfunc = 'sum')
print(family_debt_pt_sum)


                   debt
family_status          
civil partnership   383
divorced             84
married             923
unmarried           272
widow / widower      62


In [110]:
# Check the relationship between the values in 'family_status' and 'debt' using a pivot table aggregating on the count
family_debt_pt_count = pd.pivot_table(data = df, index = ['family_status'], values = ['debt'], aggfunc = 'count')
print(family_debt_pt_count)

                    debt
family_status           
civil partnership   4112
divorced            1179
married            12213
unmarried           2780
widow / widower      946


Pivot tables were generated to display the relationship between the values in 'family_status' and 'debt'. The first table aggregates on the sum to calculate the number of rows in each category of 'family_status' that defaulted on a loan while the second table aggregates on the count to calculate the number of total rows in each category of 'family_status'.

In [111]:
# Calculating default-rate based on family status
family_debt_pt_sum / family_debt_pt_count * 100


,debt
family_status,
civil partnership,9.314202
divorced,7.124682
married,7.557521
unmarried,9.784173
widow / widower,6.553911


The default-rate based on family status was calculated by dividing the sum pivot table by the count pivot table and multiplying the result by 100 to get the percentage of individuals within each category of 'family_status' that has ever defaulted on a loan.

**Conclusion**

Based on the results of the calculations, it appears that those who are widows/widowers have the lowest default rate, those who are married or divorced have slightly higher default rates, and those who are either in civil partnerships or are unmarried have the highest default rates. This information reveals that individuals who marry tend to have lower default rates than those who do not. This trend may be due to the fact that marriage leads to having dual-income and also affords some financial benefits by way of federal taxes that are not given to those in civil partnerships. Even in the aftermath of divorce, assets are often divided equally and widows/widowers often inherit fair sums of money from their deceased spouses which adds to their ability to pay off existing debt. However, particularly for the unmarried, there are fewer legal factors at play that help to ensure financial commitment and security for an individual which may negatively affect an individual's ability to repay a loan on time.

**Is there a correlation between income level and paying back on time?**

In [112]:
# Check the relationship between the values in 'income_category' and 'debt' using a pivot table aggregating on the sum
income_debt_pt_sum = pd.pivot_table(data = df, index = ['income_category'], values = ['debt'], aggfunc = 'sum')
print(income_debt_pt_sum)


                 debt
income_category      
1                 425
2                 503
3                 416
4                 380


In [113]:
# Check the relationship between the values in 'income_category' and 'debt' using a pivot table aggregating on the count
income_debt_pt_count = pd.pivot_table(data = df, index = ['income_category'], values = ['debt'], aggfunc = 'count')
print(income_debt_pt_count)


                 debt
income_category      
1                5308
2                5631
3                4983
4                5308


Pivot tables were generated to display the relationship between the values in 'income_category' and 'debt'. The first table aggregates on the sum to calculate the number of rows in each category of 'income_category' that defaulted on a loan while the second table aggregates on the count to calculate the number of total rows in each category of 'income_category'.

In [114]:
# Calculating default-rate based on income level
income_debt_pt_sum / income_debt_pt_count * 100


,debt
income_category,
1,8.006782
2,8.932694
3,8.348385
4,7.159005


The default-rate based on income category was calculated by dividing the sum pivot table by the count pivot table and multiplying the result by 100 to get the percentage of individuals within each category of 'income_category' that has ever defaulted on a loan.

**Conclusion**

Based on the calculations, it appears that those in the highest income quartile (4) have the lowest default rates, followed by those in the lowest quartile (1), second highest quartile (3), and second lowest (2). Overall, individuals with the highest income tend to default less than those in lower income quartiles. This trend may be due to the fact that those with more money often have more opportunities for income generation and more funds that are readily available for use to pay off debt than those with lower income who may have little surplus funds, especially if they are making just enough to cover their regular expenses.

**How does credit purpose affect the default rate?**

In [115]:
# Check the relationship between the values in 'purpose_category' and 'debt' using a pivot table aggregating on the sum
purpose_debt_pt_sum = pd.pivot_table(data = df, index = ['purpose_category'], values = ['debt'], aggfunc = 'sum')
print(purpose_debt_pt_sum)


                  debt
purpose_category      
car                397
education          369
real estate        777
wedding            181


In [116]:
# Check the relationship between the values in 'income_category' and 'debt' using a pivot table aggregating on the count
purpose_debt_pt_count = pd.pivot_table(data = df, index = ['purpose_category'], values = ['debt'], aggfunc = 'count')
print(purpose_debt_pt_count)


                   debt
purpose_category       
car                4258
education          3970
real estate       10703
wedding            2299


Pivot tables were generated to display the relationship between the values in 'purpose_category' and 'debt'. The first table aggregates on the sum to calculate the number of rows in each category of 'purpose_category' that defaulted on a loan while the second table aggregates on the count to calculate the number of total rows in each category of 'purpose_category'.

In [117]:
# Calculating default-rate based on purpose category
purpose_debt_pt_sum / purpose_debt_pt_count * 100


,debt
purpose_category,
car,9.323626
education,9.294710
real estate,7.259647
wedding,7.872988


The default-rate based on purpose category was calculated by dividing the sum pivot table by the count pivot table and multiplying the result by 100 to get the percentage of individuals within each category of 'purpose_category' that has ever defaulted on a loan.

**Conclusion**

Based on the above results, those whose purpose is to purchase a car or finance their education tend to have higher default rates than those whose purpose is to invest in real estate or finance a wedding. Those in the real estate category had the lowest default rate while those in the car category had the highest default rates. This may be due to the fact that real estate investments often lead to income generation and appreciation in value, and  weddings indicate that an individual will likely start having a joint-income with their spouse that gives them more financial capacity to pay off loans on time. However, the purchase of a car does not give good financial returns as it depreciates each year and regularly costs additional funds for maintenance and gas, and investing in education will often lead to an individual initially having to work less and earn less to accommodate their studies, both scenarios making it difficult to repay loans on time.

# General Conclusion 

To reiterate, the overall purpose of this project was to determine whether a customer's marital status and the number of children in their family will affect a customer's risk of defaulting on a loan. Prior the formal analysis, the initial hypotheses formulated were the following:

1) Married customers will have a lower risk of defaulting on a loan than unmarried customers

2) Customers with no children will have a lower risk of defaulting on a loan than customers with one or more children

Based on the results of the analysis, it appears that these two hypotheses were shown to be correct as married customers do have a lower risk of defaulting than unmarried customers and customers with no children have a lower risk of defaulting than customers with children.

To get to these main conclusions, the data was first processed and cleaned by dropping duplicate values, fixing register errors, dropping erroneous artifacts in the data, and filling in missing values in the 'total_income' and 'days_employed' columns based on median values calculated from non-missing data. The final processed dataset 'df' contained 21,230 rows and was utilized to proceed in the analysis. Prior to testing the correlations between defaulting on a loan and various other variables, the values in the 'purpose' and 'total_income' columns were categorized for ease of analysis. Upon completing the analysis, these are the final conclusions that were drawn:

**1) Is there a correlation between having children and paying back on time?**

Yes, individuals with children have higher default rates than those without children.

**2) Is there a correlation between family status and paying back on time?**

Yes, individuals who marry tend to have lower default rates than those who do not.

**3) Is there a correlation between income level and paying back on time?**

Yes, individuals with the highest income tend to default less than those in lower income quartiles. 

**4) How does credit purpose affect the default rate?**

Yes, those whose purpose is to purchase a car or finance their education tend to have higher default rates than those whose purpose is to invest in real estate or finance a wedding.
